In [ ]:
import threading
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.contrib.slim as slim
import scipy.signal
from skimage.color import rgb2gray
%matplotlib inline
import gym
import os

from random import choice
from time import sleep
from time import time
from AC_Network import *
from Worker import *

In [ ]:
max_episode_length = 10000
gamma = .99 
s_size = 7056 
a_size = 3 
load_model = False
model_path = './model'

In [ ]:
import os
tf.reset_default_graph()

if not os.path.exists(model_path):
    os.makedirs(model_path)
    
#Create a directory to save episode playback gifs to
if not os.path.exists('./frames'):
    os.makedirs('./frames')


global_episodes = tf.Variable(0,dtype=tf.int32,name='global_episodes',trainable=False)
trainer = tf.train.RMSPropOptimizer(learning_rate=7e-4, decay=0.99, epsilon=0.1)
master_network = AC_Network(s_size,a_size,'global',None) 
num_workers = multiprocessing.cpu_count()
workers = []

for i in range(num_workers):
    workers.append(Worker(i,s_size,a_size,trainer,model_path,global_episodes))
saver = tf.train.Saver(max_to_keep=5)

with tf.Session() as sess:
    coord = tf.train.Coordinator()
    if load_model == True:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess,ckpt.model_checkpoint_path)
    else:
        sess.run(tf.global_variables_initializer())
        
    worker_threads = []
    for worker in workers:
        worker_work = lambda: worker.work(max_episode_length,gamma,master_network,sess,coord,saver)
        t = threading.Thread(target=(worker_work))
        t.start()
        worker_threads.append(t)
    gs = 0
    while not coord.should_stop():
        s = time()
        sleep(10)
        gs1 = sess.run(global_episodes)
        print("Episodes", gs1, 'one for ', (time()-s)/(gs1-gs))
        gs = gs1
    coord.join(worker_threads)